<a href="https://colab.research.google.com/github/wisrovi/SAM/blob/master/Red%20Neuronal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
raiz = '/content/drive/'

#API google colab con google drive
from google.colab import drive
drive.mount(raiz)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [39]:
a = !ls "/content/drive/My Drive/Redes Neuronales"
if len(a)>0:
  if str(a[0]).index('cannot access') > 0:
    print("El directorio no existe, creando el directorio")
    !mkdir "/content/drive/My Drive/Redes Neuronales"
else:
  print("El directorio esta vacio")

El directorio esta vacio


In [48]:
!ls "/content/drive/My Drive/Redes Neuronales" 

'/content/drive/My Drive/Redes Neuronales':

'/content/drive/My Drive/Redes Neuronales':


In [52]:
!git clone https://github.com/wisrovi/SAM "/content/drive/My Drive/Redes Neuronales"

Cloning into '/content/drive/My Drive/Redes Neuronales'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 27 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (27/27), done.


In [53]:
!ls "/content/drive/My Drive/Redes Neuronales"

 deploy.prototxt.txt   'Red Neuronal.ipynb'
 example_opencv3-2.py   res10_300x300_ssd_iter_140000.caffemodel
 OpenCV_3.py	        samantha.apk


In [57]:
from OpenCV_3 import OpenCV_3

ModuleNotFoundError: ignored